<a href="https://colab.research.google.com/github/Swayamprakashpatel/HPLC/blob/main/HPLC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import pandas_datareader as web
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from google.colab import files
import time as tm

In [67]:
df = pd.read_csv('/content/Data.csv')
df = pd.DataFrame(df)
df = df.iloc[:,:]

X1 = df.iloc[:, 11:13]
X1 = X1.div(20).round(2) 

X2 = df.iloc[:, 13:]
X = [X1,X2]
X = pd.concat(X, axis=1)

Y_Cat = df.iloc[:, 4:7]
Y_Num = df.iloc[:,7:11]
X.head(),Y_Cat.head(), Y_Num.head()
print(X.shape, Y_Cat.shape, Y_Num.shape)

(1108, 1764) (1108, 3) (1108, 4)


In [74]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')

enc.fit(Y_Cat)
OneHotEncoder(handle_unknown='ignore')
enc.categories_

Y_Cat_OH = enc.transform(Y_Cat).toarray() #Categorical Data
print(Y_Cat_OH,Y_Cat_OH.shape)

print(Y_Cat_OH.shape)

Y2 = Y_Cat_OH

[[1. 0. 0. 1. 1. 0.]
 [0. 1. 1. 0. 1. 0.]
 [1. 0. 0. 1. 1. 0.]
 ...
 [1. 0. 1. 0. 0. 1.]
 [1. 0. 0. 1. 0. 1.]
 [0. 1. 1. 0. 1. 0.]] (1108, 6)
(1108, 6)


In [69]:
print(Y2,Y2.shape[1])

[[1. 0. 0. 1. 1. 0.]
 [0. 1. 1. 0. 1. 0.]
 [1. 0. 0. 1. 1. 0.]
 ...
 [1. 0. 1. 0. 0. 1.]
 [1. 0. 0. 1. 0. 1.]
 [0. 1. 1. 0. 1. 0.]] 6


In [70]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

X_train, X_val_and_test, Y2_train, Y2_val_and_test = train_test_split(X, Y2, test_size=0.01,random_state = 42 )
X_val, X_test, Y2_val, Y2_test = train_test_split(X_val_and_test, Y2_val_and_test, test_size=0.5, random_state= 42)

X_train = np.asarray(X_train).astype(np.int64)
X_val = np.asarray(X_val).astype(np.int64)
X_test = np.asarray(X_test).astype(np.int64)
Y2_train = np.asarray(Y2_train).astype(np.int64)
Y2_val = np.asarray(Y2_val).astype(np.int64)
Y2_test = np.asarray(Y2_test).astype(np.int64)

In [75]:
print(X_train.shape)

(1096, 1764)


In [71]:
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
filepath = '/content/drive/MyDrive/Model_DE/Model.hdf5'
 
checkpoint = [tf.keras.callbacks.ModelCheckpoint(filepath, monitor='accuracy', mode='max', save_best_only=True, Save_weights_only = False, verbose = 1), 
              tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=225, verbose =2)]

output_nodes = Y2.shape[1]
print(output_nodes)
model = tf.keras.Sequential([
                             tf.keras.layers.Dense(1764, activation='relu', input_shape=(1764,)),
                             tf.keras.layers.Dense(512, activation='relu'),
                             tf.keras.layers.Dense(512, activation='relu'),
                             tf.keras.layers.Dense(output_nodes, activation= 'Softmax')])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.8), loss=tf.keras.losses.CategoricalCrossentropy(from_logits = False), metrics=['accuracy'])
hist = model.fit(X_train, Y2_train, epochs= 2000, callbacks=[checkpoint],validation_data=(X_val, Y2_val), batch_size= 150)
model.evaluate(X_test, Y2_test)
 
Y2_train_predict = np.round(model.predict(X_train))
Y2_train_l = tf.argmax(Y2_train, axis = 1)
Y2_train_predict_l = tf.argmax(Y2_train_predict, axis =1)
import sklearn.metrics as skm
cm = skm.multilabel_confusion_matrix(Y2_train_l, Y2_train_predict_l)
print(cm)
print( skm.classification_report(Y2_train_l, Y2_train_predict_l))
 
train_acc = max(hist.history['accuracy'])
val_acc = max(hist.history['val_accuracy'])
train_loss = min(hist.history['loss'])
val_loss = min(hist.history['val_loss'])
print('Training Accuracy is')
print(train_acc)
print('Validation Accuracy is')
print(val_acc)
print('Training loss is')
print(train_loss)
print('Validation loss is')
print(val_loss)

6
Epoch 1/2000
7/8 [=========================>....] - ETA: 0s - loss: nan - accuracy: 0.3800
Epoch 1: accuracy improved from -inf to 0.38686, saving model to /content/drive/MyDrive/Model_DE/Model.hdf5
8/8 [==============================] - 1s 128ms/step - loss: nan - accuracy: 0.3869 - val_loss: nan - val_accuracy: 0.5000
Epoch 2/2000
7/8 [=========================>....] - ETA: 0s - loss: nan - accuracy: 0.5524
Epoch 2: accuracy improved from 0.38686 to 0.55474, saving model to /content/drive/MyDrive/Model_DE/Model.hdf5
8/8 [==============================] - 1s 100ms/step - loss: nan - accuracy: 0.5547 - val_loss: nan - val_accuracy: 0.5000
Epoch 3/2000
7/8 [=========================>....] - ETA: 0s - loss: nan - accuracy: 0.5590
Epoch 3: accuracy did not improve from 0.55474
8/8 [==============================] - 1s 84ms/step - loss: nan - accuracy: 0.5547 - val_loss: nan - val_accuracy: 0.5000
Epoch 4/2000
7/8 [=========================>....] - ETA: 0s - loss: nan - accuracy: 0.5505


In [ ]:
t1 = model.predict(X_train[0:5,:])
print(t1)
 
t = np.round(model.predict(X_train))
print(t)
 
Y_prediction = enc.inverse_transform(t)
print(Y1.head(),Y_prediction, Y_prediction.shape)


[[1.71530455e-01 8.16830218e-01 6.65873110e-01 3.60201508e-01
  1.99705362e-04 2.29805708e-04 6.58282876e-01 3.73521060e-01
  2.51263380e-04 2.94816494e-03 1.98728012e-05 8.19228371e-05
  3.87079358e-01 4.11924448e-05 2.22253799e-03 4.13147867e-01
  2.75138021e-03 2.40655124e-01 8.58038664e-04 1.06510520e-03
  6.77317381e-04 7.55854368e-01 2.60028243e-03 2.05155629e-05
  2.05650061e-01 1.74760818e-04 9.11428579e-05 9.99983191e-01
  7.51295805e-01 2.40557075e-01 8.49283970e-05 2.69323587e-04
  1.92046165e-04 1.62856122e-05 1.26387904e-05]
 [4.54753081e-06 9.99998808e-01 2.39379663e-08 9.99999285e-01
  1.53970058e-21 4.55224325e-17 9.99996603e-01 1.51708662e-06
  5.24412413e-15 5.02116237e-10 1.25044635e-20 1.46101262e-18
  9.99990106e-01 7.79245744e-18 7.07244983e-12 3.66711191e-07
  1.82847085e-10 8.12716561e-09 7.39361304e-14 1.21551515e-14
  1.19079825e-18 2.98875129e-05 3.23215318e-05 9.99941826e-01
  7.37296613e-11 6.56344163e-12 6.15678307e-15 1.00000000e+00
  1.00000000e+00 1.538

In [ ]:
X.to_csv('Drug_1.csv')